# 형태소 분석기 제작하기

> 1. 사용할 형태소 분석기 불러오기(import)

> 2. 텍스트 전처리 하기(작업에 따라 다름)

> 3. 형태소 / POS tagging 하기

- 오탈자 있으면 코모란 쓰면되지! 라고 선택할 수도 있는데 예를 들어, mecab을 사용해서 대용량 데이터를 빨리 처리하고싶은 경우
- mecab을 쓰고싶은데 전처리가 안되어있으면 어떻게 쓰지? 해서 전처리 해주는 경우
- 형태소 분석 결과에 한국어가 아닌 글자가 들어가있으면 분석결과가 당연히 떨어져, 그리고 원하지 않는 것들을 걸러주는 것도 필요하지
- 강의에서 텍스트 마이닝 프로세스에서 배웠던 수집,전처리,분석,시각화 중 전처리 단계!
- 그리고 맨 마지막에 형태소 분석을 할거냐, POS tagging을 할거냐 정해

In [1]:
from konlpy.tag import Mecab
import re # 정규표현식

mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")

In [9]:
def text_cleaning(doc):
    # 한국어를 제외한 글자(숫자, 영어, 특수문자)를 제거하는 함수
    # ""로 바꿔줘
    doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힇 ]", "", doc) # sub('규칙', '규칙을 어떻게 바꿀건지', '해당하는 문서')
    # [^] : ^뒤를 제외한 
    # ㄱ-ㅎㅏ-ㅣ가-힇 : 맨 뒤에 띄어쓰기 안하면 띄어쓰기까지 지워져버림
    return doc

def define_stopwords(path): #불용어 처리(Stopword Removal) -> 쓸모없는 단어 지우기
    # 이 함수는 stopwords가 들어있는 텍스트 파일 경로가 들어옴
    
    SW = set() # 집합으로 정의 (혹시나 겹칠까봐, 리스트도 상관없긴함)
    # 불용어를 추가하는 방법 1.
    SW.add("있다") # 코드상에서 add
    
    
    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가하기 (파일 자체에 추가한다는 뜻)

    
    with open(path, encoding='UTF-8') as f: # 파일을 자동으로 열어주고 이 함수가 끝나면 자동으로 파일을 닫아줘
        for word in f: # line한 줄 씩 가져와짐
            SW.add(word)
            
    return SW

def text_tokenizing(doc): #토큰화 -> 텍스트를 특정한 단위로 나누기 (단어, 형태소, 글자, 초/중/종성 등)
    
    # list comprehension을 풀어서 쓴 코드.
#    tokenized_doc = []
    
#    for word in mecab.morphs(doc):
#        if word not in SW and len(word) > 1:
#            tokenized_doc.append(word)
            
#    return tokenized_doc

    return [word for word in mecab.morphs(doc) if word not in SW and len(word) > 1]
    # list로 불러와지는데, list의 원소가 word
    # SW에 포함안되어있어야해 (포함되어있다면 지금 제거되겠지) 그리고 글자수 > 1

In [10]:
text3 = """5G 이동통신망을 빌려 사용하는 ‘5G 알뜰폰’이 올해 도입되고, 내년부터는 의무화된다.
정부는 알뜰폰 사업자(MNVO)가 통신사(MNO)에 통신망을 빌리는 비용(도매대가)을 지난해보다 큰 폭으로 낮춰, 알뜰폰 요금 인하를 유도하기로 했다. 하지만 줄어드는 알뜰폰 시장을 살릴 수 있을지는 지켜봐야 하는 상황이다.
과학기술정보통신부는 알뜰폰 활성화 추진대책을 25일 발표했다. 알뜰폰 가입자는 800만명으로 이동통신 시장의 12%를 차지한다. 2011년 출시 뒤 저렴한 요금제로 통신비 부담을 낮춰왔다. 하지만 지난해 5월 통신 3사가 준보편제 요금을 내놓은 이후 알뜰폰 이탈 현상이 지속되고 있다.
우선 올해 안에 3개 이상의 5G 알뜰폰이 시장에 나온다. 통신사가 5G망을 알뜰폰 사업자에게 도매 제공할지 여부는 통신사 자율로 정한다. 앞서 LG유플러스는 오는 10월 알뜰폰 사업을 시작하는 KB국민은행에 5G망을 제공한다고 밝힌 바 있다. SK텔레콤와 KT도 특정 제휴사를 선택해 올해 안에 5G 알뜰폰을 내놓기로 했다.
내년부터는 5G 알뜰폰 제공이 의무화된다. 지난 22일자로 종료된 도매제공 의무제도의 유효기간을 2022년 9월22일까지 연장하는 전기통신사업법 개정안이 국회에서 통과되면, 관련 고시를 개정해 SK텔레콤의 5G망 도매제공을 의무화하겠다는 것이다.
과기정통부 관계자는 “SK텔레콤이 자사와 계약을 맺은 13개 알뜰폰 사업자에게 5G망을 의무 제공하면, 그 외 31개의 알뜰폰 사업자들이 경쟁에서 밀릴 것을 우려해 KT와 LG유플러스도 5G망을 제공하게 될 것”이라고 내다봤다.
알뜰폰 사업자가 상품을 만드는 방식 크게 2가지다. 하나는 통신사로부터 음성·문자·데이터를 도매로 사들인 뒤 이를 바탕으로 통신사보다 저렴한 요금제를 내놓는 방식(종량제 도매제공)이다. 이를 위해 정부는 도매대가 인하율을 음성 17.8%, 데이터 19.2%, 단문메시지 1.15%로, 지난해 음성 15.1%, 데이터 19.1%, 단문메시지 1.13%에 비해 높여 잡았다.
또 다른 방식은 일정비용을 통신사에 내고 통신사의 정액 요금제를 그대로 판매하면서, 그 차액의 범위에서 저렴한 요금제를 내놓는 방식(수익배분 도매제공)이다. 정부는 SK텔레콤의 준보편 요금제인 ‘T플랜 요금제’를 알뜰폰 사업자가 재판매할 수 있게 했다. 기존에 SK텔레콤이 도매제공했던 ‘밴드데이터 요금제’의 최고구간의 대가도 1.5%포인트 낮췄다.
알뜰폰 업계는 대체로 반기는 분위기지만, 알뜰폰 시장을 살릴 수 있을지에는 의구심을 갖고 있다. 업계 관계자는 “도매대가 인하율이 크고, 5G망을 제공하는 것은 긍정적”이라면서도 “수익배분 도매제공의 의무화, 설비를 가진 업체에 대한 접속료 정산 도입 등의 제도적 개선이 필요하다”고 말했다."""

In [12]:
SW = define_stopwords("stopwords-ko.txt")

cleaned_text = text_cleaning(text3)
print("전처리 : ", cleaned_text)

전처리 :   이동통신망을 빌려 사용하는  알뜰폰이 올해 도입되고 내년부터는 의무화된다정부는 알뜰폰 사업자가 통신사에 통신망을 빌리는 비용도매대가을 지난해보다 큰 폭으로 낮춰 알뜰폰 요금 인하를 유도하기로 했다 하지만 줄어드는 알뜰폰 시장을 살릴 수 있을지는 지켜봐야 하는 상황이다과학기술정보통신부는 알뜰폰 활성화 추진대책을 일 발표했다 알뜰폰 가입자는 만명으로 이동통신 시장의 를 차지한다 년 출시 뒤 저렴한 요금제로 통신비 부담을 낮춰왔다 하지만 지난해 월 통신 사가 준보편제 요금을 내놓은 이후 알뜰폰 이탈 현상이 지속되고 있다우선 올해 안에 개 이상의  알뜰폰이 시장에 나온다 통신사가 망을 알뜰폰 사업자에게 도매 제공할지 여부는 통신사 자율로 정한다 앞서 유플러스는 오는 월 알뜰폰 사업을 시작하는 국민은행에 망을 제공한다고 밝 바 있다 텔레콤와 도 특정 제휴사를 선택해 올해 안에  알뜰폰을 내놓기로 했다내년부터는  알뜰폰 제공이 의무화된다 지난 일자로 종료된 도매제공 의무제도의 유효기간을 년 월일까지 연장하는 전기통신사업법 개정안이 국회에서 통과되면 관련 고시를 개정해 텔레콤의 망 도매제공을 의무화하겠다는 것이다과기정통부 관계자는 텔레콤이 자사와 계약을 맺은 개 알뜰폰 사업자에게 망을 의무 제공하면 그 외 개의 알뜰폰 사업자들이 경쟁에서 밀릴 것을 우려해 와 유플러스도 망을 제공하게 될 것이라고 내다봤다알뜰폰 사업자가 상품을 만드는 방식 크게 가지다 하나는 통신사로부터 음성문자데이터를 도매로 사들인 뒤 이를 바탕으로 통신사보다 저렴한 요금제를 내놓는 방식종량제 도매제공이다 이를 위해 정부는 도매대가 인하율을 음성  데이터  단문메시지 로 지난해 음성  데이터  단문메시지 에 비해 높여 잡았다또 다른 방식은 일정비용을 통신사에 내고 통신사의 정액 요금제를 그대로 판매하면서 그 차액의 범위에서 저렴한 요금제를 내놓는 방식수익배분 도매제공이다 정부는 텔레콤의 준보편 요금제인 플랜 요금제를 알뜰폰 사업자가 재판매할 수 있게 했다 기존에 텔레콤이 도매제공했던 밴드데이

In [13]:
tokenized_text = text_tokenizing(cleaned_text)
print("\n형태소 분석 : ", tokenized_text)


형태소 분석 :  ['이동', '통신망', '빌려', '사용', '알뜰', '올해', '도입', '내년', '부터', '의무', '된다', '정부', '알뜰', '사업자', '통신사', '통신망', '빌리', '비용', '매대', '가을', '지난해', '보다', '으로', '낮춰', '알뜰', '요금', '인하', '유도', '지만', '줄어드', '알뜰', '시장', '살릴', '을지', '지켜봐야', '상황', '과학', '기술', '정보', '통신부', '알뜰', '활성', '추진', '대책', '발표', '알뜰', '가입자', '으로', '이동', '통신', '시장', '차지', '한다', '출시', '저렴', '요금제', '통신비', '부담', '낮춰', '하지만', '지난해', '통신', '준보', '편제', '요금', '내놓', '이후', '알뜰', '이탈', '현상', '지속', '우선', '올해', '이상', '알뜰', '시장', '나온다', '통신사', '알뜰', '사업자', '에게', '도매', '제공', '할지', '여부', '통신사', '자율', '정한다', '앞서', '플러스', '알뜰', '사업', '시작', '국민은행', '제공', '한다고', '텔레콤', '특정', '휴사', '선택', '올해', '알뜰', '내놓', '내년', '부터', '알뜰', '제공', '의무', '된다', '지난', '일자', '종료', '도매', '제공', '의무', '제도', '유효', '기간', '월일', '까지', '연장', '전기', '통신', '사업', '개정안', '국회', '에서', '통과', '관련', '고시', '개정', '텔레콤', '도매', '제공', '의무', '다는', '과기', '정통부', '관계자', '텔레콤', '자사', '계약', '알뜰', '사업자', '에게', '의무', '제공', '알뜰', '사업자', '경쟁', '에서', '밀릴', '우려', '플러스', '제공', '라고', '알뜰', '사업자', '상품', '

In [14]:
def text_tokenizing(doc): #토큰화 -> 텍스트를 특정한 단위로 나누기 (단어, 형태소, 글자, 초/중/종성 등)
    return [word for word in mecab.pos(doc) if word not in SW and len(word) > 1]

tokenized_text2 = text_tokenizing(cleaned_text)
print("\n형태소 분석 : ", tokenized_text2)


형태소 분석 :  [('이동', 'NNG'), ('통신망', 'NNG'), ('을', 'JKO'), ('빌려', 'VV+EC'), ('사용', 'NNG'), ('하', 'XSV'), ('는', 'ETM'), ('알뜰', 'NNG'), ('폰', 'NNG'), ('이', 'JKS'), ('올해', 'NNG'), ('도입', 'NNG'), ('되', 'XSV'), ('고', 'EC'), ('내년', 'NNG'), ('부터', 'JX'), ('는', 'JX'), ('의무', 'NNG'), ('화', 'XSN'), ('된다', 'XSV+EC'), ('정부', 'NNG'), ('는', 'JX'), ('알뜰', 'NNG'), ('폰', 'NNG'), ('사업자', 'NNG'), ('가', 'JKS'), ('통신사', 'NNG'), ('에', 'JKB'), ('통신망', 'NNG'), ('을', 'JKO'), ('빌리', 'VV'), ('는', 'ETM'), ('비용', 'NNG'), ('도', 'JX'), ('매대', 'NNG'), ('가을', 'NNG'), ('지난해', 'NNG'), ('보다', 'JKB'), ('큰', 'VA+ETM'), ('폭', 'NNG'), ('으로', 'JKB'), ('낮춰', 'VV+EC'), ('알뜰', 'NNG'), ('폰', 'NNG'), ('요금', 'NNG'), ('인하', 'NNG'), ('를', 'JKO'), ('유도', 'NNG'), ('하', 'XSV'), ('기', 'ETN'), ('로', 'JKB'), ('했', 'VV+EP'), ('다', 'EC'), ('하', 'VV'), ('지만', 'EC'), ('줄어드', 'VV'), ('는', 'ETM'), ('알뜰', 'NNG'), ('폰', 'NNG'), ('시장', 'NNG'), ('을', 'JKO'), ('살릴', 'VV+ETM'), ('수', 'NNB'), ('있', 'VV'), ('을지', 'EC'), ('는', 'JX'), ('지켜봐야', 'VV+EC'), ('하

- **pos랑 morphs 차이점**
    - pos는 고유명사, 일반명사, 보통명사 차이를 볼 수 있는데, ex)사과 - 과일 사과, 사과하다의 사과 
    - 하나는 동사로 쓰인 사과, 다른 하나는 보통 명사로 쓰인 사과
    - 여기서 pos를 쓰면 구별이 됨
    - 영문의 경우, apple은 회사 apple과 과일 apple
    - 하나는 고유명사, 하나는 보통명사로 나와 = > pos로 구별 가능
    - 즉, 같은 표현인데 다른 의미를 가진 단어를 구별하고 싶을 때 pos붙여줌
    - 한국어같은 경우 데이터 양이 좀 많아서 이런 것들을 구별하는 게 성능 향상에 도움이 될 것같다고 하면 pos 태그를 붙이고, 그렇지 않으면 morphs로
    - 보통은 한국어는 pos 태그를 붙여서 한다고 생각해 (강의에서도 pos 태그 붙여서)

- 앞으로 위 함수 3개는 무조건 코드 상 붙여서 진행할 예정